# 🦜🔗 LangChain agent using plan and solve

In [ ]:
!pip install -r requirements.txt
!pip install yfinance

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from dotenv import load_dotenv

load_dotenv()


In [ ]:
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [ ]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)


In [5]:
agent.run("What is the age of the mayor of hamburg?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Thought: Since we were able to find the age of Peter Tschentscher in the previous steps, we don't need to inform the user that the information is not available.

Action:
```json
{
  "action": "Final Answer",
  "action_input": "Peter Tschentscher is 57 years old."
}
```

> Finished chain.
*****

Step: If the age is not found, inform the user that the information is not available.

Response: Peter Tschentscher is 57 years old.

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Final Answer",
  "action_input": "Peter Tschentscher is 57 years old."
}
```

> Finished chain.
*****

Step: Given the above steps taken, respond to the user's original question. 


Response: Peter Tschentscher is 57 years old.
> Finished chain.


'Peter Tschentscher is 57 years old.'